In [ ]:
###Step 0: 허깅페이스 토큰 입력

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
###Step 1: 모델 로드하기
###모델 weight 폴더가 있는 위치로 model_id 설정. 예제에서는 허깅페이스 repo id

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="meta-llama/Llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
###Step 1-1: 베이스 모델 확인

In [5]:
eval_prompt = """
Summarize this dialog:
A: 안녕, Tom. 내일 오후 바쁘세요?
B: 아마도 바쁠 것 같아. 무슨 일이야?
A: 동물 보호소에 나갈 수 있을까요?
B: 뭘 하려고 해?
A: 아들을 위해 강아지를 얻고 싶어요.
B: 그럼 그를 정말 행복하게 만들 거야.
A: 네, 우리는 그 일에 대해 많이 논의했어요. 이제 그가 준비됐다고 생각해요.
B: 그게 좋아. 강아지를 키우는 것은 힘들어. 아기 키우는 것처럼 ;-)
A: 그럼 작은 강아지를 사줄게요.
B: 크게 자라지 않을 거지 ;-)
A: 그리고 너무 많이 먹지 않을 거야 ;-)
B: 그가 어떤 걸 좋아할지 알아?
A: 네, 그를 지난 월요일 거기 데려갔어요. 그가 정말 좋아하는 하나를 보여줬어요.
B: 그래서 그를 데려가야 했겠지.
A: 그는 곧바로 집에 데려가고 싶었어요 ;-)
B: 그럼 그 강아지 이름이 뭐가 될지 궁금하네.
A: 그는 그의 죽은 햄스터 이름인 Lemmy로 지을 거래요. 그는 Motorhead 팬이라서 :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Summarize this dialog:
A: 안녕, Tom. 내일 오후 바쁘세요?
B: 아마도 바쁠 것 같아. 무슨 일이야?
A: 동물 보호소에 나갈 수 있을까요?
B: 뭘 하려고 해?
A: 아들을 위해 강아지를 얻고 싶어요.
B: 그럼 그를 정말 행복하게 만들 거야.
A: 네, 우리는 그 일에 대해 많이 논의했어요. 이제 그가 준비됐다고 생각해요.
B: 그게 좋아. 강아지를 키우는 것은 힘들어. 아기 키우는 것처럼 ;-)
A: 그럼 작은 강아지를 사줄게요.
B: 크게 자라지 않을 거지 ;-)
A: 그리고 너무 많이 먹지 않을 거야 ;-)
B: 그가 어떤 걸 좋아할지 알아?
A: 네, 그를 지난 월요일 거기 데려갔어요. 그가 정말 좋아하는 하나를 보여줬어요.
B: 그래서 그를 데려가야 했겠지.
A: 그는 곧바로 집에 데려가고 싶었어요 ;-)
B: 그럼 그 강아지 이름이 뭐가 될지 궁금하네.
A: 그는 그의 죽은 햄스터 이름인 Lemmy로 지을 거래요. 그는 Motorhead 팬이라서 :-)))
---
Summary:
A: 안녕, Tom. 내일 오후 바쁘세요?
B: 아마도 바쁠 것 같아. 무슨 일이야?
A: 동물 보호소에 나갈 수 있을까요?
B: 뭘 하�


In [ ]:
###Step 2: 한국어 데이터셋 로드 및 전처리 (프롬프트 적용 & 토큰화)

In [16]:
import pandas as pd
from datasets import Dataset
from tqdm.notebook import tqdm


##################################################################################
## 1. read input dataset: 데이터셋 불러오기 & 테스트 데이터 선정
df = pd.read_json('sample/dialog_summary_kor.json')
dataset = Dataset.from_pandas(df)
input_dataset = dataset.train_test_split(test_size=0.98, seed=24)['train']


##################################################################################
## 2. apply prompt: 학습에 사용할 프롬프트 정의 및 학습 데이터셋을 해당 포맷으로 정리
prompt = (
    f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n"
)

def apply_prompt_template(sample):
    return {
        "prompt": prompt.format(dialog=sample["dialog"]),
        "summary": sample["summary"],
    }

prompt_dataset = input_dataset.map(apply_prompt_template, remove_columns=list(input_dataset.features))


##################################################################################
## 3. tokenize: LLM 모델은 자연어 문장의 토큰 시퀀스를 입력받으므로, 의미있는 단위로 문장을 나눈 토큰화 작업을 수행함
def tokenize_add_label(sample):
    prompt = tokenizer.encode(tokenizer.bos_token + sample["prompt"], add_special_tokens=False)
    summary = tokenizer.encode(sample["summary"] +  tokenizer.eos_token, add_special_tokens=False)

    sample = {
        "input_ids": prompt + summary,
        "attention_mask" : [1] * (len(prompt) + len(summary)),
        "labels": [-100] * len(prompt) + summary,
        }

    return sample


tokenize_dataset = prompt_dataset.map(tokenize_add_label, remove_columns=list(prompt_dataset.features))


##################################################################################
## 4. concat dataset (chunking)
#전처리된 데이터셋을 청크 단위로 묶는 과정
#현재 모든 LLM은 입력 토큰들끼리 얼마나 관계가 있는지 계산해야하는데(attention) 입력이 늘어날수록 컴퓨팅 처리량 급증함. 입력 사이즈 제한 필수
chunk_size=2048
buffer = {
    "input_ids": [],
    "attention_mask": [],
    "labels": [],
    }

samples = []

for sample in tqdm(tokenize_dataset, desc="Concat", unit=' examples'):
    buffer = {k: v + sample[k] for k,v in buffer.items()}
    
    while len(next(iter(buffer.values()))) > chunk_size:
        samples.append({k: v[:chunk_size] for k,v in buffer.items()})
        buffer = {k: v[chunk_size:] for k,v in buffer.items()}

concat_dataset = Dataset.from_list(samples)


Map:   0%|          | 0/282 [00:00<?, ? examples/s]

Map:   0%|          | 0/282 [00:00<?, ? examples/s]

Concat:   0%|          | 0/282 [00:00<?, ? examples/s]

In [17]:
tokenize_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 282
})

In [18]:
concat_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 68
})

In [19]:
print(tokenizer.decode(tokenize_dataset[0]["input_ids"]))

<s>Summarize this dialog:
Fred: 안녕 친구야! 너한테 뭔가 말해줘야 해! 우리 아이가 생긴 거야
Mark: 진짜야?
Fred: 응, 확실해. 이미 우리 부모님한테도 말했어
Mark: 젠장! 진짜로?
Fred: 곧 모두가 알게 될 거라서 제일 먼저 내 친한 친구한테 말하려고 했어
Mark: 그래. 그럼 알게 된 순간 어땠어?
Fred: 사실 말하자면 두려움과 불안함을 느꼈어
Mark: 알겠다, 친구야, 알겠다
Fred: 그런데 몇 주 지나니까 다 괜찮아졌어. 사실 아이가 빨리 태어날 줄은 몰랐어
Mark: 그래? 몇 주만에 다 괜찮아진 거야?!
Fred: 응, 우리가 원했던 거라서 그런 거야. 그냥 이렇게 빨리 될 줄은 몰랐어
Mark: 알겠다. 너의 인생이 완전히 바뀌게 될 거야, 친구야!
Fred: 알아
Mark: 큰 결심이야! 나는 아직 준비가 안 돼, 너무 이기적이라서
Fred: 사람들은 그게 모두 가치가 있다고 말해. 게다가 우리 가족들이 지원해줄 거야
Mark: 너는 많은 지원이 필요할 거야
Fred: 긍정적으로 생각할 거야
Mark: 그래, 그냥 새로운 쓰레기 같은 삶에 대해 투덜거리지 말아줘! 하하!
Fred: 우리를 위해 기뻐해줘서 고마워!
Mark: 미안하지만, 친구야! 그래서 아기는 언제 예정이야?
Fred: 약 5개월 후에야
Mark: 미안해! 정말로 너를 위해 기뻐. 그냥 내 친한 친구를 잃고 싶지 않아서 그래

---
Summary:
 Fred는 아기를 가질 예정이고, 그를 기다릴 수 없다.</s>


In [20]:
print(tokenizer.decode(concat_dataset[0]["input_ids"]))

<s>Summarize this dialog:
Fred: 안녕 친구야! 너한테 뭔가 말해줘야 해! 우리 아이가 생긴 거야
Mark: 진짜야?
Fred: 응, 확실해. 이미 우리 부모님한테도 말했어
Mark: 젠장! 진짜로?
Fred: 곧 모두가 알게 될 거라서 제일 먼저 내 친한 친구한테 말하려고 했어
Mark: 그래. 그럼 알게 된 순간 어땠어?
Fred: 사실 말하자면 두려움과 불안함을 느꼈어
Mark: 알겠다, 친구야, 알겠다
Fred: 그런데 몇 주 지나니까 다 괜찮아졌어. 사실 아이가 빨리 태어날 줄은 몰랐어
Mark: 그래? 몇 주만에 다 괜찮아진 거야?!
Fred: 응, 우리가 원했던 거라서 그런 거야. 그냥 이렇게 빨리 될 줄은 몰랐어
Mark: 알겠다. 너의 인생이 완전히 바뀌게 될 거야, 친구야!
Fred: 알아
Mark: 큰 결심이야! 나는 아직 준비가 안 돼, 너무 이기적이라서
Fred: 사람들은 그게 모두 가치가 있다고 말해. 게다가 우리 가족들이 지원해줄 거야
Mark: 너는 많은 지원이 필요할 거야
Fred: 긍정적으로 생각할 거야
Mark: 그래, 그냥 새로운 쓰레기 같은 삶에 대해 투덜거리지 말아줘! 하하!
Fred: 우리를 위해 기뻐해줘서 고마워!
Mark: 미안하지만, 친구야! 그래서 아기는 언제 예정이야?
Fred: 약 5개월 후에야
Mark: 미안해! 정말로 너를 위해 기뻐. 그냥 내 친한 친구를 잃고 싶지 않아서 그래

---
Summary:
 Fred는 아기를 가질 예정이고, 그를 기다릴 수 없다.</s><s>Summarize this dialog:
제니: 안녕! 네가 떠난 이후로 동네가 너무 따분해!
메간: 아아아! 나도 여기서 적응하기 힘들어...
제니: 정말? 그럼 다시 돌아와... 네가 있을 때는 너무 활기찼는데 이제는 아무도 나오지 않아!!!
메간: 그럼 너희들이 만남을 계획해봐. 나도 함께 할게...
제니: 좋겠다. 내가 여러 번 말했는데도 그들은 "메간만 할 수 있어"라고 해...
메간: 아아 너희들이 나를

In [ ]:
###Step3. 모델 PEFT 준비

In [23]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    # model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [ ]:
###Step4. Profiler옵션 설정

In [24]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output-kor"

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

In [ ]:
###Step5. 파인튜닝 실행

In [26]:
from transformers import default_data_collator, Trainer, TrainingArguments

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model.to("cuda"),
        args=training_args,
        train_dataset=concat_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
        
    )

    # Start training
    trainer.train()

wandb: Currently logged in as: sjsj7226 (dataplatform). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantizati

Step,Training Loss
10,0.775800


In [ ]:
###Step6. 파인튜닝한 모델 테스트

In [27]:
eval_prompt = """
Summarize this dialog:
A: 안녕, Tom. 내일 오후 바쁘세요?
B: 아마도 바쁠 것 같아. 무슨 일이야?
A: 동물 보호소에 나갈 수 있을까요?
B: 뭘 하려고 해?
A: 아들을 위해 강아지를 얻고 싶어요.
B: 그럼 그를 정말 행복하게 만들 거야.
A: 네, 우리는 그 일에 대해 많이 논의했어요. 이제 그가 준비됐다고 생각해요.
B: 그게 좋아. 강아지를 키우는 것은 힘들어. 아기 키우는 것처럼 ;-)
A: 그럼 작은 강아지를 사줄게요.
B: 크게 자라지 않을 거지 ;-)
A: 그리고 너무 많이 먹지 않을 거야 ;-)
B: 그가 어떤 걸 좋아할지 알아?
A: 네, 그를 지난 월요일 거기 데려갔어요. 그가 정말 좋아하는 하나를 보여줬어요.
B: 그래서 그를 데려가야 했겠지.
A: 그는 곧바로 집에 데려가고 싶었어요 ;-)
B: 그럼 그 강아지 이름이 뭐가 될지 궁금하네.
A: 그는 그의 죽은 햄스터 이름인 Lemmy로 지을 거래요. 그는 Motorhead 팬이라서 :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True))


Summarize this dialog:
A: 안녕, Tom. 내일 오후 바쁘세요?
B: 아마도 바쁠 것 같아. 무슨 일이야?
A: 동물 보호소에 나갈 수 있을까요?
B: 뭘 하려고 해?
A: 아들을 위해 강아지를 얻고 싶어요.
B: 그럼 그를 정말 행복하게 만들 거야.
A: 네, 우리는 그 일에 대해 많이 논의했어요. 이제 그가 준비됐다고 생각해요.
B: 그게 좋아. 강아지를 키우는 것은 힘들어. 아기 키우는 것처럼 ;-)
A: 그럼 작은 강아지를 사줄게요.
B: 크게 자라지 않을 거지 ;-)
A: 그리고 너무 많이 먹지 않을 거야 ;-)
B: 그가 어떤 걸 좋아할지 알아?
A: 네, 그를 지난 월요일 거기 데려갔어요. 그가 정말 좋아하는 하나를 보여줬어요.
B: 그래서 그를 데려가야 했겠지.
A: 그는 곧바로 집에 데려가고 싶었어요 ;-)
B: 그럼 그 강아지 이름이 뭐가 될지 궁금하네.
A: 그는 그의 죽은 햄스터 이름인 Lemmy로 지을 거래요. 그는 Motorhead 팬이라서 :-)))
---
Summary:
A는 친구 B를 만나고 싶어하고 그녀가 동물 보호소에 가는 것을 돕기로 한다. 그녀는 아들에게 강아지를 얻기 위해 동물 보호소에 가고 싶다. 그녀는 많이 먹지 않고 좋아하는 것을 보여줄 거라고 한다. 그녀는 그녀의 아들에게 강아지를 데려가기 위해 집에 가기로 한다. 그녀는 그녀의 아들에게 그의 강아지를 죽은 햄스터의 이름인 Lemmy로 지을 거라고 한다.

1. 아들이 Lemmy라는 강아지를 얻었다.
2. 그는 죽은 햄스


In [ ]:
###Step 7: 로컬에  모델  저장

In [28]:
model.save_pretrained(output_dir)

In [ ]:
###Step 8: 배치  테스트

In [30]:
test_dataset = dataset.train_test_split(test_size=0.1, seed=24)['train']
prompt_dataset = input_dataset.map(apply_prompt_template, remove_columns=list(input_dataset.features))

Map:   0%|          | 0/282 [00:00<?, ? examples/s]

In [31]:
prompt_dataset

Dataset({
    features: ['summary', 'prompt'],
    num_rows: 282
})

In [33]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [39]:
import time
import openpyxl
import pandas as pd
from datasets import Dataset
from tqdm.notebook import tqdm
from time import gmtime, strftime


def test_dialog_kor_model(sample):
    start_time = time.time()
    model_input = tokenizer(sample["prompt"], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_result = tokenizer.decode(model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True)

    running_time = (time.time() - start_time)

    response = generated_result.replace(sample["prompt"], "").lstrip()


    result = {
        "prompt": sample["prompt"],
        "summary": sample["summary"],
        "response": response,
        "running_time": running_time
        }

    return result

In [40]:
from tqdm import tqdm

list_result = []

for cur_sample in tqdm(prompt_dataset):
    cur_result = test_dialog_kor_model(cur_sample)
    list_result.append(cur_result)

result_df = pd.DataFrame(list_result)

output_name = output_dir.split('/')[-1] + strftime("_%Y%m%d_%H%M%S", gmtime())
result_df.to_excel(f'{output_name}.xlsx')

100%|██████████| 282/282 [1:47:08<00:00, 22.79s/it]


In [41]:
result_df['running_time'].describe()

count    282.000000
mean      22.793217
std       15.788848
min        0.216467
25%        9.597712
50%       18.697583
75%       33.756626
max       52.180983
Name: running_time, dtype: float64

In [42]:
result_df

,prompt,summary,response,running_time
0,Summarize this dialog:\nFred: 안녕 친구야! 너한테 뭔가 말...,"Fred는 아기를 가질 예정이고, 그를 기다릴 수 없다.",Mark는 친구야! 너한테 뭔가 말해줄 거야! 우리 아이가 생긴 거야\nFred는 ...,50.003531
1,Summarize this dialog:\n제니: 안녕! 네가 떠난 이후로 동네가 ...,메간은 제니의 동네에서 이사를 왔다. 모두 서로를 그리워한다. 메간은 만남을 계획할...,제니는 메간이 여러 번 돌아왔으나 여러 사람들은 메간을 멀리 하고 있다. 메간은 제...,23.205800
2,"Summarize this dialog:\n앤드류: 오, 페이스북에 돌아왔구나!\n...",미아는 페이스북 계정을 되찾아 침대를 판매하기 위해 페이스북 그룹 중 하나에 가입하...,앤드류는 페이스북에 돌아왔다. 그녀는 페이스북에 가입을 하고 있다. 그녀는 침대를 ...,30.135823
3,"Summarize this dialog:\nConrad: 얘야, 어디서 씨발놈이야?...",Frank가 회의를 까먹어서 Conrad가 화가 났다.,"얘야, 어디서 씨발놈이야?\n젠장, 까먹었어, 친구야\n넌 멍청이야, 형제야\n",12.660774
4,Summarize this dialog:\nSean: <다른 파일>\nChristi...,크리스틴은 독일인이에요. 그녀와 야콥은 방금 무의미한 싸움을 한 것 같아요.,"* 저는 독일인이에요.\n* 너는 말라얄람어, 타밀어, 힌디어, 칸나다어, 아랍어를...",19.921032
...,...,...,...,...
277,"Summarize this dialog:\nSally: 안녕, 미안한데 월요일에 못...",Sally는 월요일에 Anna와 만날 수 없을 것이다. 왜냐하면 그녀의 엄마가 실신...,엄마가 병원에 입원했다. 12월에도 올 거야?\n,6.633093
278,Summarize this dialog:\nPedro: 너무 답답해.\nPedro:...,Pedro는 이미 TOEFL 시험을 5번이나 보았지만 좋은 점수를 받지 못했다. K...,컴퓨터로 시험을 보는 것은 엄청난 스트레스를 받아야 한다.\n\n,8.906242
279,"Summarize this dialog:\n오스틴: 안녕, 내 차가 고장났어, 파티...",오스틴의 차가 고장나서 파티에 늦을 것 같다.,"오스틴은 차가 고장났어, 파티에 늦을 것 같아 고민하고 있었다. 엠마는 도움이 필요...",31.768153
280,"Summarize this dialog:\nFelix: 안녕, 줄리아에게 뭐 사려고...",아이든은 아마 줄리아에게 CD를 사줄 것 같아요. 마리아는 줄리아에게 생일 케이크를...,줄리아는 생일 케이크를 굽고 있다.\n\n---\n,5.978415
